In [1]:
# Import data from CSV files
from csv import DictReader

with open('_data/fluidized_bed.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows_fb = [r for r in reader]
csv_file.close()

with open('_data/batch_reactor.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows_br = [r for r in reader]
csv_file.close()

with open('_data/tubular_reactor.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows_tr = [r for r in reader]
csv_file.close()

rows_all = rows_fb[:]
rows_all.extend(rows_br)
rows_all.extend(rows_tr)

INPUT_VARS = [
    'Reaction Time (s)',
    'Temperature (C)',
    'Particle size (mm)',
    'Moisture',
    'Ash',
    'C',
    'H',
    'N',
    'O'
]

TARGET_VARS = [
    'Oil',
    'Char',
    'Gas'
]

In [2]:
# Obtain spearman coefficients for every input/target relationship
from typing import List, Tuple
from scipy.stats import spearmanr

def get_inp_tar(rows: List[dict], inp: str, tar) -> Tuple[List[float]]:

    inputs = []
    targets = []
    for row in rows:
        try:
            _i = float(row[inp])
            _t = float(row[tar])
            inputs.append(_i)
            targets.append(_t)
        except:
            continue
    return (inputs, targets)

results = []
for inp in INPUT_VARS:
    for tar in TARGET_VARS:

        inputs_fb, targets_fb = get_inp_tar(rows_fb, inp, tar)
        inputs_br, targets_br = get_inp_tar(rows_br, inp, tar)
        inputs_tr, targets_tr = get_inp_tar(rows_tr, inp, tar)
        inputs_all, targets_all = get_inp_tar(rows_all, inp, tar)

        corr_fb, p_fb = spearmanr(inputs_fb, targets_fb)
        corr_br, p_br = spearmanr(inputs_br, targets_br)
        corr_tr, p_tr = spearmanr(inputs_tr, targets_tr)
        corr_all, p_all = spearmanr(inputs_all, targets_all)

        results.append({
            'Input Parameter': inp,
            'Yield': tar,
            'Rho (FB)': corr_fb,
            'P-value (FB)': p_fb,
            'Rho (BR)': corr_br,
            'P-value (BR)': p_br,
            'Rho (TR)': corr_tr,
            'P-value (TR)': p_tr,
            'Rho (All)': corr_all,
            'P-value (All)': p_all
        })

In [3]:
# Dump spearman results to CSV file
from csv import DictWriter

headers = [
    'Input Parameter',
    'Yield',
    'Rho (FB)',
    'P-value (FB)',
    'Rho (BR)',
    'P-value (BR)',
    'Rho (TR)',
    'P-value (TR)',
    'Rho (All)',
    'P-value (All)'
]

with open('_results/spearman_results.csv', 'w', encoding='utf8') as csv_file:
    writer = DictWriter(csv_file, headers, delimiter=',', lineterminator='\n')
    writer.writeheader()
    writer.writerows(results)
csv_file.close()

In [4]:
# Highlight relationships with low (< 0.01) p-value (i.e., statistically significant)

for res in results:
    if res['P-value (FB)'] < 0.01:
        print(f"FB: {res['Input Parameter']} | {res['Yield']} | {res['Rho (FB)']} | {res['P-value (FB)']}")
    if res['P-value (BR)'] < 0.01:
        print(f"BR: {res['Input Parameter']} | {res['Yield']} | {res['Rho (BR)']} | {res['P-value (BR)']}")
    if res['P-value (TR)'] < 0.01:
        print(f"TR: {res['Input Parameter']} | {res['Yield']} | {res['Rho (TR)']} | {res['P-value (TR)']}")
    if res['P-value (All)'] < 0.01:
        print(f"All: {res['Input Parameter']} | {res['Yield']} | {res['Rho (All)']} | {res['P-value (All)']}")

BR: Reaction Time (s) | Oil | 0.6450290234230088 | 1.232976618207201e-05
All: Reaction Time (s) | Oil | -0.3914838778281055 | 1.3329673554815417e-06
FB: Reaction Time (s) | Char | 0.396930619968646 | 0.0016893685331950052
All: Reaction Time (s) | Char | 0.3698710354115384 | 5.4547887170163066e-06
TR: Reaction Time (s) | Gas | 0.7938987726782729 | 7.705630613706507e-11
All: Reaction Time (s) | Gas | 0.226519078515349 | 0.006519203690156338
FB: Temperature (C) | Char | -0.8344806651338886 | 6.605110384269438e-17
BR: Temperature (C) | Char | -0.8746946043273244 | 7.106856890288747e-13
TR: Temperature (C) | Char | -0.4218871697724732 | 0.0018385177258764206
All: Temperature (C) | Char | -0.6553607079538746 | 6.848186546121846e-20
FB: Temperature (C) | Gas | 0.7291131888236582 | 2.6843767391153935e-11
BR: Temperature (C) | Gas | 0.9062818460291913 | 4.980257204006057e-15
TR: Temperature (C) | Gas | 0.6466677190907956 | 2.2303852975929174e-07
All: Temperature (C) | Gas | 0.7160829984452666 |